In [46]:
import torch
import torch.nn as nn 
import numpy as np
import pandas as pd

In [47]:
# Import raw dataset
df = pd.read_csv('data.csv')


In [ ]:
# train-test split manually, keeping times unshuffled
split = False

if split:
    mark = round(0.7 * df.shape[0])
    train = df[:mark]
    test = df[mark:]
    print(test.shape)

    train.to_csv("train.csv", index=False)
    test.to_csv("test.csv", index=False)

In [54]:
# converting date column to pandas datetime type
df["date"] = df['date'].astype(str)
df["date"] = pd.to_datetime(df['date'])

In [55]:
# exploration: see number of entries for each date
times = df["date"].unique()
temp = df.date.value_counts()

In [63]:
# exploration: see number of entries for each street
df["from_to_pair"].value_counts()

CENTRAL PARK WEST-8 Avenue Line-8 Avenue Line                         52285
CONNECTOR-Queens Boulevard-Queens Boulevard                           46913
7 AVENUE-Broadway/ 7 Avenue Line-Broadway/ 7 Avenue Line              29088
LONG ISLAND EXPRESSWAY-Dead End-Dead end                              23994
WILLIAMSBURG BRIDGE-East River Shoreline West-East River Shoreline    22183
                                                                      ...  
BRONX BOULEVARD-Dead End-Bronx River Shoreline                           24
WEST 39 STREET-Lincoln Tunnel Exit-11 Avenue                             24
CROOKE AVENUE-St Pauls Place-Brighton Line                               24
44 AVENUE-72 Street-Amtrak-connecting Rail Line                          24
STATEN ISLAND RAILWAY LINE-Ferry Terminal Drive-Borough Place            24
Name: from_to_pair, Length: 10806, dtype: int64

In [91]:
# isolate one street to train on
isolated_top_street = df[df["from_to_pair"] == "CENTRAL PARK WEST-8 Avenue Line-8 Avenue Line"]
isolated_top_street['date'] = isolated_top_street['date'].apply(lambda x: x.value)
isolated_top_street.drop('from_to_pair', axis=1, inplace=True)
arr = torch.tensor(isolated_top_street.values).long()

C:\Users\austi\AppData\Local\Temp\ipykernel_23872\4284110750.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isolated_top_street['date'] = isolated_top_street['date'].apply(lambda x: x.value)
c:\Users\austi\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [93]:
arr

tensor([[1413279000000000000,                  68],
        [1413279900000000000,                  75],
        [1413280800000000000,                  59],
        ...,
        [1508109300000000000,                  99],
        [1508110200000000000,                  94],
        [1508111100000000000,                  81]])

In [60]:
# isolate one datetime to train on
temp3 = df[df['date'] == pd.to_datetime("2013-10-28 11:00:00+00:00")]
temp3.value_counts()


date                       from_to_pair                                                   Vol
2013-10-28 11:00:00+00:00  EDISON AV-DUDLEY AV-WELLMAN AV                                 7      2
                           TROY ST-ARTHUR KILL RD-DALEHAM ST                              1      2
                           ARKANSAS AV-LEXINGTON LA-ISLINGTON ST/GIFFORDS LA              1      2
                           GREAVES AV-DALEHAM ST-CROMER ST                                3      2
                           DALEHAM ST-GIFFORDS LA-MILES AV                                2      2
                                                                                                ..
                           CENTRAL PARK W-W 109 ST-FRED DOUGLASS CIR                      96     1
                           CENTRAL PARK W-CPW 72 APPR/W 72 ST-W 73 ST                     124    1
                           CENTRAL PARK W-COLUMBUS CIR-W 61 ST                            181    1
               

In [ ]:
# ON HOLD iterate through dataset
if False:
    for i, unique_time in enumerate(times):
        #print(unique_time, type(unique_time))
        temp = df['date'] == unique_time
        if unique_time == :
            temp.to_csv("temp.csv", index=False)
            print("done")
    
    

In [85]:
arr = arr.type(torch.LongTensor)

In [87]:
type(arr)

torch.Tensor

In [94]:
hidden_size = 64
num_layers = 2
rnn = nn.LSTM(input_size=2, hidden_size=hidden_size, num_layers=num_layers, bidirectional=True)

D = 2
input = arr
h0 = torch.randn(D * num_layers, hidden_size)
c0 = torch.randn(D * num_layers, hidden_size)

output, (hn, cn) = rnn(input, (h0, c0))

RuntimeError: expected scalar type Long but found Float

In [2]:
rnn = nn.RNN(10, 20, 2)
h0 = torch.randn(2, 3, 20)
output, hn = rnn(input, h0)

In [3]:
output

tensor([[[-7.7515e-01, -6.2970e-01, -4.1391e-02, -9.0979e-01, -6.0172e-02,
           9.0428e-01,  8.5135e-01,  1.3005e-01, -9.8243e-02, -4.3599e-01,
           5.7951e-02, -1.8930e-01, -5.4880e-01, -6.7055e-01, -5.3618e-01,
          -6.7847e-01, -4.2819e-02, -5.0814e-01, -3.5833e-01, -4.3119e-02],
         [-6.9912e-01,  4.7982e-01,  3.6744e-01, -5.4635e-01, -3.0642e-01,
           1.9530e-01, -4.2088e-01,  3.0594e-01,  6.3326e-01, -4.7822e-01,
           7.4071e-02, -7.0710e-01,  2.2135e-01, -3.7941e-01, -4.9712e-01,
           4.6727e-01,  1.8716e-01,  1.3151e-01, -4.3147e-01, -8.4226e-01],
         [-8.3876e-01, -7.4943e-03, -1.0124e-01, -5.4867e-01,  3.3398e-01,
           9.6304e-01, -4.3037e-01,  8.4618e-01, -4.8463e-01, -2.0326e-01,
          -5.4331e-01,  2.4944e-01, -5.4894e-01, -3.9072e-01, -3.1265e-01,
          -3.2740e-01, -4.5764e-01, -5.3447e-02,  3.5748e-01,  2.2720e-01]],

        [[ 2.3301e-01,  4.9354e-01,  6.0046e-02, -4.4865e-01, -5.1506e-01,
          -3.7624e-0